<a href="https://colab.research.google.com/github/1kaiser/proccessing_sketches/blob/main/Render%F0%9F%8E%A6p5js.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <div align="left"> <a href="https://editor.p5js.org/" target="_parent"><img width="50" alt="Avatar" src="https://pbs.twimg.com/profile_images/502135348663578624/-oslcYof_400x400.png" alt="p5.js is a JavaScript library"/> </a></div>Code 



In [8]:
%%writefile p5.txt
/*
 * @name Flocking
 * @description Demonstration of <a href="http://www.red3d.com/cwr/">Craig Reynolds' "Flocking" behavior</a>.<br>
 * (Rules: Cohesion, Separation, Alignment.)<br>
 * From <a href="http://natureofcode.com">natureofcode.com</a>.
 */
let boids = [];
let fr = 30;
function setup() {
  createCanvas(1920, 1080,WEBGL);
frameRate(fr);
  // Add an initial set of boids into the system
  for (let i = 0; i < 100; i++) {
    boids[i] = new Boid(random(width), random(height));
  }
}
let maxFrames = 1000;
let saveIt = 1;
let frames = 0;
function draw() {
  background(51);
  // Run all the boids
  for (let i = 0; i < boids.length; i++) {
    boids[i].run(boids);
  }
  // save code starts here
  if(saveIt && frames < maxFrames ){
   let num = nf(frames, 4, 0);
   let filename = num + ".png"; 
   save(filename);
   frames++;
  }
}

// Boid class
// Methods for Separation, Cohesion, Alignment added
class Boid {
  constructor(x, y) {
    this.acceleration = createVector(0, 0);
    this.velocity = p5.Vector.random2D();
    this.position = createVector(x, y);
    this.r = 3.0;
    this.maxspeed = 3;    // Maximum speed
    this.maxforce = 0.05; // Maximum steering force
  }

  run(boids) {
    this.flock(boids);
    this.update();
    this.borders();
    this.render();
  }
  
  // Forces go into acceleration
  applyForce(force) {
    this.acceleration.add(force);
  }
  
  // We accumulate a new acceleration each time based on three rules
  flock(boids) {
    let sep = this.separate(boids); // Separation
    let ali = this.align(boids);    // Alignment
    let coh = this.cohesion(boids); // Cohesion
    // Arbitrarily weight these forces
    sep.mult(2.5);
    ali.mult(1.0);
    coh.mult(1.0);
    // Add the force vectors to acceleration
    this.applyForce(sep);
    this.applyForce(ali);
    this.applyForce(coh);
  }
  
  // Method to update location
  update() {
    // Update velocity
    this.velocity.add(this.acceleration);
    // Limit speed
    this.velocity.limit(this.maxspeed);
    this.position.add(this.velocity);
    // Reset acceleration to 0 each cycle
    this.acceleration.mult(0);
  }
  
  // A method that calculates and applies a steering force towards a target
  // STEER = DESIRED MINUS VELOCITY
  seek(target) {
    let desired = p5.Vector.sub(target, this.position); // A vector pointing from the location to the target
    // Normalize desired and scale to maximum speed
    desired.normalize();
    desired.mult(this.maxspeed);
    // Steering = Desired minus Velocity
    let steer = p5.Vector.sub(desired, this.velocity);
    steer.limit(this.maxforce); // Limit to maximum steering force
    return steer;
  }
  
  // Draw boid as a circle
  render() {
    fill(127, 127);
    stroke(200);
    ellipse(this.position.x, this.position.y, 16, 16);
  }
  
  // Wraparound
  borders() {
    if (this.position.x < -this.r) this.position.x = width + this.r;
    if (this.position.y < -this.r) this.position.y = height + this.r;
    if (this.position.x > width + this.r) this.position.x = -this.r;
    if (this.position.y > height + this.r) this.position.y = -this.r;
  }
  
  // Separation
  // Method checks for nearby boids and steers away
  separate(boids) {
    let desiredseparation = 25.0;
    let steer = createVector(0, 0);
    let count = 0;
    // For every boid in the system, check if it's too close
    for (let i = 0; i < boids.length; i++) {
      let d = p5.Vector.dist(this.position, boids[i].position);
      // If the distance is greater than 0 and less than an arbitrary amount (0 when you are yourself)
      if ((d > 0) && (d < desiredseparation)) {
        // Calculate vector pointing away from neighbor
        let diff = p5.Vector.sub(this.position, boids[i].position);
        diff.normalize();
        diff.div(d); // Weight by distance
        steer.add(diff);
        count++; // Keep track of how many
      }
    }
    // Average -- divide by how many
    if (count > 0) {
      steer.div(count);
    }
  
    // As long as the vector is greater than 0
    if (steer.mag() > 0) {
      // Implement Reynolds: Steering = Desired - Velocity
      steer.normalize();
      steer.mult(this.maxspeed);
      steer.sub(this.velocity);
      steer.limit(this.maxforce);
    }
    return steer;
  }
  
  // Alignment
  // For every nearby boid in the system, calculate the average velocity
  align(boids) {
    let neighbordist = 50;
    let sum = createVector(0, 0);
    let count = 0;
    for (let i = 0; i < boids.length; i++) {
      let d = p5.Vector.dist(this.position, boids[i].position);
      if ((d > 0) && (d < neighbordist)) {
        sum.add(boids[i].velocity);
        count++;
      }
    }
    if (count > 0) {
      sum.div(count);
      sum.normalize();
      sum.mult(this.maxspeed);
      let steer = p5.Vector.sub(sum, this.velocity);
      steer.limit(this.maxforce);
      return steer;
    } else {
      return createVector(0, 0);
    }
  }
  
  // Cohesion
  // For the average location (i.e. center) of all nearby boids, calculate steering vector towards that location
  cohesion(boids) {
    let neighbordist = 50;
    let sum = createVector(0, 0); // Start with empty vector to accumulate all locations
    let count = 0;
    for (let i = 0; i < boids.length; i++) {
      let d = p5.Vector.dist(this.position, boids[i].position);
      if ((d > 0) && (d < neighbordist)) {
        sum.add(boids[i].position); // Add location
        count++;
      }
    }
    if (count > 0) {
      sum.div(count);
      return this.seek(sum); // Steer towards the location
    } else {
      return createVector(0, 0);
    }
  }  
}



Overwriting p5.txt


# 📜 to Process Render the <a href="https://editor.p5js.org/" target="_parent"><img width="30" alt="Avatar" src="https://pbs.twimg.com/profile_images/502135348663578624/-oslcYof_400x400.png" alt="p5.js is a JavaScript library"/> </a>       code for the 🎞️Frames

## downloading puppeteer <a href="https://github.com/puppeteer/puppeteer" target="_parent"><img width="30" alt="Avatar" src="https://user-images.githubusercontent.com/10379601/29446482-04f7036a-841f-11e7-9872-91d1fc2ea683.png" alt="Puppeteer is a Node library which provides a high-level API to control Chrome or Chromium"/> </a>

In [2]:
!npm install -g npm
!npm install puppeteer bluebird
!apt-get update
!apt install chromium-chromedriver


/tools/node/bin/npm -> /tools/node/lib/node_modules/npm/bin/npm-cli.js
/tools/node/bin/npx -> /tools/node/lib/node_modules/npm/bin/npx-cli.js
+ npm@8.5.5
added 58 packages from 18 contributors, removed 292 packages and updated 142 packages in 5.908s

added 59 packages, and audited 60 packages in 11s

8 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https

## puppeteer code edit

In [5]:
%%writefile index.js

'use strict';

const puppeteer = require('puppeteer');
const fs = require('fs');
const bluebird = require("bluebird");


(async() => {
  const browser = await puppeteer.launch({executablePath:"/usr/lib/chromium-browser/chromium-browser", args:['--no-sandbox']});
  let withPage = (browser) => async (fn) => {
        const page = await browser.newPage();
        try {
            return await fn(page);
        } catch (err) {
            console.error(err.message);
           // await page.close();
            return false;
        } finally {
          //  await page.close();
        }
    }
    let SavingFolder = "MD"
    if (!fs.existsSync(SavingFolder)) {
        fs.mkdirSync(SavingFolder, function (err) {
            if (err)
                console.log(err);
        });
    }
    let file = '\out.txt'
    if (!fs.existsSync(file)) {
        fs.writeFileSync(file, '', function (err) {
            if (err)
                console.log(err);
        });
    }
    var darray = fs.readFileSync('\out.txt', 'utf8').split('\n');
    let QFile = '\p5.txt'
    if (!fs.existsSync(QFile)) {
        fs.writeFileSync(QFile, '', function (err) {
            if (err)
                console.log(err);
        });
    }
    var inputdata = fs.readFileSync('\p5.txt', 'utf8').replace(/\/\*[\s\S]*?\*\/|([^:]|^)\/\/.*$/gm,'').replace(/(\r\n|\r|\n|\t)/g, '')
    let urls = ['https://editor.p5js.org/']
    const titles = [];
    bluebird.map(urls, async (url) => {
        if (!darray.join().includes(url)) {
            return withPage(browser)(async (page) => {
                let completed = []
                let data = false;
                let attempts = 0;
                while (data == false && attempts < 1) {
                    await page.goto(url, {
                        waitUntil: "networkidle0",
                    });
                    await page._client.send("Page.setDownloadBehavior", {
                        behavior: "allow",
                        downloadPath: SavingFolder,
                    })
                    let selectorBe= '#root > div > div.RootPage-sc-1si1i3e-0.fPOANA > main > div > div.Pane.vertical.Pane2 > div > div.Pane.vertical.Pane1 > div > div.Pane.horizontal.Pane1 > section > article > div > div.CodeMirror-scroll > div.CodeMirror-sizer > div > div > div > div.CodeMirror-code > div > pre'
                    if(page.waitForSelector(selectorBe)){data=true}
                    await page.click(selectorBe)
                    await page.keyboard.down('ControlLeft');
                    await page.keyboard.press('KeyA');
                    await page.keyboard.press('Delete');
                    await page.keyboard.up('ControlLeft');
                    await page.click(selectorBe)
                    for (let index = 0; index < inputdata.length; index++) {
                        if (inputdata[index]==')'||inputdata[index]=='}'||inputdata[index]==']') {
                            await page.keyboard.press('ArrowRight');
                        }else{
                        await page.type( selectorBe, inputdata[index], { delay: 10 }); // Types slower, like a user
                        }
                        console.log('Loading🖊️📝...remaing time:',(inputdata.length-index)*10/1000,' s⌚')
                    }
                    console.log('executing 💻')
                    page.waitForTimeout(1000)
                    await page.click('#root > div > div.RootPage-sc-1si1i3e-0.fPOANA > div > button.toolbar__play-button')
                    attempts += 1;
                    if (data != false) {
                        completed.push(url.concat('\n'))
                       // fs.appendFileSync('out.txt', completed.toString())
                    }
                }
            }).then((r) => ({
                result: r
            }), (e) => ({
                error: e
            }));
        }
    }, {
        concurrency: 3 //<< increase to go parallel tabs using bluebird library
    });

})()

Overwriting index.js


## *RUN*

In [10]:
!node index.js

Loading🖊️📝...remaing time: 35.99  s⌚
Loading🖊️📝...remaing time: 35.98  s⌚
Loading🖊️📝...remaing time: 35.97  s⌚
Loading🖊️📝...remaing time: 35.96  s⌚
Loading🖊️📝...remaing time: 35.95  s⌚
Loading🖊️📝...remaing time: 35.94  s⌚
Loading🖊️📝...remaing time: 35.93  s⌚
Loading🖊️📝...remaing time: 35.92  s⌚
Loading🖊️📝...remaing time: 35.91  s⌚
Loading🖊️📝...remaing time: 35.9  s⌚
Loading🖊️📝...remaing time: 35.89  s⌚
Loading🖊️📝...remaing time: 35.88  s⌚
Loading🖊️📝...remaing time: 35.87  s⌚
Loading🖊️📝...remaing time: 35.86  s⌚
Loading🖊️📝...remaing time: 35.85  s⌚
Loading🖊️📝...remaing time: 35.84  s⌚
Loading🖊️📝...remaing time: 35.83  s⌚
Loading🖊️📝...remaing time: 35.82  s⌚
Loading🖊️📝...remaing time: 35.81  s⌚
Loading🖊️📝...remaing time: 35.8  s⌚
Loading🖊️📝...remaing time: 35.79  s⌚
Loading🖊️📝...remaing time: 35.78  s⌚
Loading🖊️📝...remaing time: 35.77  s⌚
Loading🖊️📝...remaing time: 35.76  s⌚
Loading🖊️📝...remaing time: 35.75  s⌚
Loading🖊️📝...remaing time: 35.74  s⌚
Loading🖊️📝...remaing time: 35.73  s⌚
Loa

# 🎥 Create Video Output

In [11]:
#create video 1
!ffmpeg \
-framerate 30 \
-pattern_type glob \
-i '/content/MD/*.png' \
input0.mp4
##create vodeo 2
#!ffmpeg \
#-framerate 24 \
#-pattern_type glob \
#-i '/content/MD/*.png' \
#input1.mp4
##merge video 1 and video 2
#!ffmpeg \
#-i input0.mp4 -i input1.mp4 \
#-filter_complex \
#"[0:v][1:v]vstack=inputs=2" \
#finalOutput.mp4
##add sound of main video to final output video
#!ffmpeg \
#-i 'finalOutput.mp4' -i 'm.mp4' \
#-c:v copy -c:a copy -map 0:v:0 -map 1:a:0 \
#OUTPUT_FILE.mp4


ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

# 📂 zipping

In [ ]:
!zip -r /content/20220313.zip /content/MD

# <a href="https://drive.google.com/" target="_parent"><img width="25" alt="Avatar" src="//ssl.gstatic.com/images/branding/product/1x/drive_2020q4_48dp.png" alt="google drive"/> </a> upload to google drive 

In [ ]:
!pip install gshell

get the key  paste the key

* `output_directory_id` : Google drive folder url `https://drive.google.com/drive/u/0/folders/1ph2v7i3Qm8U6KVPh1DEJ1UCXRKVMYEY0 ` use ***1ph2v7i3Qm8U6KVPh1DEJ1UCXRKVMYEY0***  <<<<as *directory_id*

In [ ]:
!gshell init
output_directory_id = '1ph2v7i3Qm8U6KVPh1DEJ1UCXRKVMYEY0' #@param ['xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'] {allow-input: true}

!gshell upload --parent '{output_directory_id}' --recursive /content/20220313.zip